# Data Cleaning & Feature Engineering

## Pull in the data

In [1]:
# import libraries
import pandas as pd
import numpy as np
import os
import json

Pull in the data from the Cresci-2017 dataset, set the appropriate target and concatenate data frames

genuine_accounts.csv has some extra columns relating to when data was crawled, these will need to be rmeoved when joining we union the two dfs.

In [2]:
data_dir = './data/'
fake_follower_df = pd.read_csv(data_dir+'fake_followers.csv')
fake_follower_df['target'] = 1
genuine_accounts_df = pd.read_csv(data_dir+'genuine_accounts.csv')
# remove extra columns in genuine_accounts.csv
genuine_accounts_df.drop(['timestamp', 'crawled_at', 'test_set_1', 'test_set_2'], inplace=True, axis =1)
genuine_accounts_df['target'] = 0
df = pd.concat([fake_follower_df, genuine_accounts_df])
df.head()

,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,created_at,url,...,is_translator,follow_request_sent,protected,verified,notifications,description,contributors_enabled,following,updated,target
0,80479674,YI YUAN,yi_twitts,29,19,255,1,0,Wed Oct 07 03:19:21 +0000 2009,http://www.jycondo.com,...,NaN,NaN,NaN,NaN,NaN,real estate sales,NaN,NaN,2013-06-12 18:38:35,1
1,82487179,Marcos Perez C,marcos_peca,1408,208,866,138,0,Wed Oct 14 23:40:17 +0000 2009,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-06-12 18:38:35,1
2,105830531,curti lorenzo,curtilorenzo,39,59,962,8,0,Sun Jan 17 16:46:52 +0000 2010,http://www.valcavargna.com/,...,NaN,NaN,NaN,NaN,NaN,le corna del capro scappato dal gregge s'infil...,NaN,NaN,2013-06-12 18:38:35,1
3,114488344,ruben dario toscano,gatito2710,59,7,49,4,0,Mon Feb 15 15:49:58 +0000 2010,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-06-12 18:38:35,1
4,123222267,Malek Khalaf,MalekKhalaf,987,60,521,61,1,Mon Mar 15 11:38:55 +0000 2010,http://www.facebook.com/Malek.AlBalawi,...,NaN,NaN,NaN,NaN,NaN,"MA student at JU, Interested in Juventus,Italy...",NaN,NaN,2013-06-11 17:39:44,1


In [3]:
print("""
    Dimensions
    Rows: {rows}
    Columns: {columns}
    """.format(rows=len(df), columns=len(df.columns)))


    Dimensions
    Rows: 6825
    Columns: 39
    


## Data Cleaning

The Cresci-2017 dataset was built using an older vesion of the Twitter API (2013). 
Since 2017 Twitter has deprecated a number of features, to ensure our model is forward compatible 
with the new Twitter API fields we must clean out the deprecated fields

**Note:** "screen_name" has been replaced with "username" in Twitter API v2

| cresci | Twitter API V2 |
| --- | --- |
| screen_name | username |
| friends_count | following_count |
| statuses_count | tweet_count |

In [4]:
#rename columns to reflect the current API fields
df = df.rename(columns={'screen_name':'username',
                       'friends_count': 'following_count',
                       'statuses_count': 'tweet_count'})

In [5]:
cresci_colums = df.columns[:37] # don't include upadted and target
twitter_object_colums = pd.read_csv(data_dir+'twitter_user_object.csv')['Field value'].values


In [6]:
print("""
    No. Cresci-2017 columns: {cresci}
    No. Twitter user object columns: {twitter}
    """.format(cresci=len(cresci_colums), twitter=len(twitter_object_colums)))


    No. Cresci-2017 columns: 37
    No. Twitter user object columns: 18
    


We can use set operations to find out the fields present in our training datset that are not present in Twitter API v2

In [7]:
difference = set(cresci_colums).difference(set(twitter_object_colums))
intersection = set(cresci_colums).intersection(set(twitter_object_colums))
new_api_difference = set(twitter_object_colums).difference(set(cresci_colums))
print("""
No. deprecated fields: {len_fields}
================================
Deprecated Fields:
{diff}
================================
No. intersecting fields fields: {len_fields_2}
================================
Intersection Fields:
{inter}
================================
No. new fields: {len_fields_3}
================================
New API Fields:
{diff_2}
    """.format(len_fields=len(difference), diff=difference, 
               inter=intersection, len_fields_2=len(intersection),
               diff_2 =new_api_difference, len_fields_3=len(new_api_difference)))


No. deprecated fields: 23
Deprecated Fields:
{'profile_text_color', 'contributors_enabled', 'profile_banner_url', 'profile_background_color', 'default_profile_image', 'profile_link_color', 'notifications', 'time_zone', 'profile_image_url_https', 'profile_background_image_url', 'favourites_count', 'profile_background_image_url_https', 'utc_offset', 'geo_enabled', 'default_profile', 'profile_sidebar_border_color', 'profile_sidebar_fill_color', 'lang', 'profile_use_background_image', 'following', 'is_translator', 'follow_request_sent', 'profile_background_tile'}
No. intersecting fields fields: 14
Intersection Fields:
{'url', 'username', 'tweet_count', 'listed_count', 'location', 'followers_count', 'profile_image_url', 'name', 'description', 'verified', 'id', 'following_count', 'created_at', 'protected'}
No. new fields: 4
New API Fields:
{'pinned_tweet_id', 'withheld', 'entities', 'public_metrics'}
    


Since 2017 it appears 23 fields have been deprecated, 4 fields have been added and 14 fields have remained available. We will only keep the remaining 14 fields.

In [8]:
# Only keep the intersection columns and add feature columns back in
df = df[list(intersection)+['target', 'updated']]

In [9]:
df.columns

Index(['url', 'username', 'tweet_count', 'listed_count', 'location',
       'followers_count', 'profile_image_url', 'name', 'description',
       'verified', 'id', 'following_count', 'created_at', 'protected',
       'target', 'updated'],
      dtype='object')

The boolean features "protected" and "verified" are missing a 0 value when they are not true, we need to replace these with 0's.

In [10]:
df.fillna({'protected':int(0), 'verified':0}, inplace=True)
df[['protected', 'verified']] = df[['protected', 'verified']].astype(int)
df.head()

,url,username,tweet_count,listed_count,location,followers_count,profile_image_url,name,description,verified,id,following_count,created_at,protected,target,updated
0,http://www.jycondo.com,yi_twitts,29,0,toronto,19,http://a0.twimg.com/profile_images/1832741069/...,YI YUAN,real estate sales,0,80479674,255,Wed Oct 07 03:19:21 +0000 2009,0,1,2013-06-12 18:38:35
1,NaN,marcos_peca,1408,0,NaN,208,http://a0.twimg.com/profile_images/3106285928/...,Marcos Perez C,NaN,0,82487179,866,Wed Oct 14 23:40:17 +0000 2009,0,1,2013-06-12 18:38:35
2,http://www.valcavargna.com/,curtilorenzo,39,0,San Bartolomeo Val Cavargna CO,59,http://a0.twimg.com/profile_images/1743973394/...,curti lorenzo,le corna del capro scappato dal gregge s'infil...,0,105830531,962,Sun Jan 17 16:46:52 +0000 2010,0,1,2013-06-12 18:38:35
3,NaN,gatito2710,59,0,peru,7,http://a0.twimg.com/profile_images/1174713108/...,ruben dario toscano,NaN,0,114488344,49,Mon Feb 15 15:49:58 +0000 2010,0,1,2013-06-12 18:38:35
4,http://www.facebook.com/Malek.AlBalawi,MalekKhalaf,987,1,"31.975193,35.920294",60,http://a0.twimg.com/profile_images/2861541411/...,Malek Khalaf,"MA student at JU, Interested in Juventus,Italy...",0,123222267,521,Mon Mar 15 11:38:55 +0000 2010,0,1,2013-06-11 17:39:44


### convert the created_at and updated values to a simple date

In [11]:
from datetime import datetime

df['created_at'] = df['created_at'].map(lambda x: datetime.strftime(datetime.strptime(x,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d') )
df['updated'] = df['updated'].map(lambda x: datetime.strftime(datetime.strptime(x,'%Y-%m-%d %H:%M:%S'), '%Y-%m-%d') )
df.head()

,url,username,tweet_count,listed_count,location,followers_count,profile_image_url,name,description,verified,id,following_count,created_at,protected,target,updated
0,http://www.jycondo.com,yi_twitts,29,0,toronto,19,http://a0.twimg.com/profile_images/1832741069/...,YI YUAN,real estate sales,0,80479674,255,2009-10-07,0,1,2013-06-12
1,NaN,marcos_peca,1408,0,NaN,208,http://a0.twimg.com/profile_images/3106285928/...,Marcos Perez C,NaN,0,82487179,866,2009-10-14,0,1,2013-06-12
2,http://www.valcavargna.com/,curtilorenzo,39,0,San Bartolomeo Val Cavargna CO,59,http://a0.twimg.com/profile_images/1743973394/...,curti lorenzo,le corna del capro scappato dal gregge s'infil...,0,105830531,962,2010-01-17,0,1,2013-06-12
3,NaN,gatito2710,59,0,peru,7,http://a0.twimg.com/profile_images/1174713108/...,ruben dario toscano,NaN,0,114488344,49,2010-02-15,0,1,2013-06-12
4,http://www.facebook.com/Malek.AlBalawi,MalekKhalaf,987,1,"31.975193,35.920294",60,http://a0.twimg.com/profile_images/2861541411/...,Malek Khalaf,"MA student at JU, Interested in Juventus,Italy...",0,123222267,521,2010-03-15,0,1,2013-06-11


### Remove unwanted columns
* id: this is the unique identifier for an account and is not suitable as a feature
* profile_image_url: Image classification is outside the scope of this study.

In [12]:
df.drop(['id', 'profile_image_url'], inplace=True, axis=1)

In [13]:
print("""
    Dimensions
    Rows: {rows}
    Columns: {columns}
    """.format(rows=len(df), columns=len(df.columns)))


    Dimensions
    Rows: 6825
    Columns: 14
    


# Feature engineering

## Text based features <br>
We have the below text based features in our dataset that must be engineered to be useful to our ML model. As Natural Language is ouside the scope of this study we will look to convert them to boolean values with 1 insinuating the feature is present.

**Text/String Features**
'location', 'description', 'url', 'name', 'username',

#### name_containment
A username to name containment feature (Jaccard Simmilarity) that describes whether the “username” contains the same word tokens as the “name”. The assumption here is that for fake followers these two fields will be similar as generating a different value for each field could be harder to implement. Further to this, when you sign up for Twitter the “username” suggested is often a concatenation of first and last name. An easy way around this would be to introduce some random characters into the suggested name, to account for this a containment function was used as opposed to a Levenshtein distance approach that would not appreciate this quirk.

In [14]:
def calc_name_containment(name, username):
    '''Calculates a conatinment score between the name and username of a twitter account.
    This fucntion will splite the name into token and then count how many of those tokens are
    present in the username and then return the normalised containment value
    :param name: name of account
    :param username: username of account'''
    # Make both lowercase
    name = str(name).lower()
    username = str(username).lower()
    name_list = name.split(' ')
    total = len(name_list)
    count = 0
    for token in name_list:
        # check the token is in the username string and is longer than 1 character
        if username.find(token) != -1 and len(token) > 1:
            count+=1
    if count != 0:
        containment = count/len(name_list)
        return containment
    else:
        return 0



### Test Cell

In [15]:
assert calc_name_containment('Pat Lam', 'Patbam99') == 0.5, "Should equal 0.5"

### Add new containment feature to df and drop the "name" and "user_name" columns

In [16]:
df['name_containment'] = df.apply(lambda x: calc_name_containment(x['name'], x['username']), axis=1)
df.drop(['name', 'username'], axis=1, inplace=True)

For the rest of the text features we will convert them to a boolean (1,0) dependant on wether they are present or not

In [17]:
df['url'] = df['url'].apply(lambda x: 1 if type(x)==str else 0)
df['location'] = df['location'].apply(lambda x: 1 if type(x)==str else 0)
df['description'] = df['description'].apply(lambda x: 1 if type(x)==str else 0)
df.head()

,url,tweet_count,listed_count,location,followers_count,description,verified,following_count,created_at,protected,target,updated,name_containment
0,1,29,0,1,19,1,0,255,2009-10-07,0,1,2013-06-12,0.500000
1,0,1408,0,0,208,0,0,866,2009-10-14,0,1,2013-06-12,0.333333
2,1,39,0,1,59,1,0,962,2010-01-17,0,1,2013-06-12,1.000000
3,0,59,0,1,7,0,0,49,2010-02-15,0,1,2013-06-12,0.000000
4,1,987,1,1,60,1,0,521,2010-03-15,0,1,2013-06-11,1.000000


## Numerical Features
Lastly we convert our count features to per_day ratios. This is done because a simple count can be misleading, younger accounts will tend to have less tweets, followers and friends because they have not had time to perform these actions. For example, an account may only have 10 tweets, but is only 10 days old meaning it sends 1 tweet per day conversly an old account could also have 10 tweets but be 100 days old giving us 0.1 tweets per day. in this scenario, although tweet counts are the same, it is evident the younger account tweets more regularly.

### Create account_age_days feature
account_age_days will allow us to calculate "per_day" features from the three count variables 

In [18]:
# convert the created_at and updated columns to datetime objects
df['created_at'] = pd.to_datetime(df['created_at'],format='%Y-%m-%d')
df['updated'] = pd.to_datetime(df['updated'],format='%Y-%m-%d')
# calculate the account age in days
df['account_age_days'] = (df['updated'] - df['created_at']).dt.days

### Make "per_day" features
followers_count, tweet_count and following_count will be converted to a ratio.

In [19]:
df['followers_per_day'] = round(df['followers_count']/df['account_age_days'], 4)
df['tweets_per_day'] = round(df['tweet_count']/df['account_age_days'], 4)
df['following_per_day'] = round(df['following_count']/df['account_age_days'], 4)
# Drop the unwanted columns
#df.drop(['followers_count', 'tweet_count', 'following_count', 'created_at', 'updated'], inplace=True, axis=1)
df.head()

,url,tweet_count,listed_count,location,followers_count,description,verified,following_count,created_at,protected,target,updated,name_containment,account_age_days,followers_per_day,tweets_per_day,following_per_day
0,1,29,0,1,19,1,0,255,2009-10-07,0,1,2013-06-12,0.500000,1344,0.0141,0.0216,0.1897
1,0,1408,0,0,208,0,0,866,2009-10-14,0,1,2013-06-12,0.333333,1337,0.1556,1.0531,0.6477
2,1,39,0,1,59,1,0,962,2010-01-17,0,1,2013-06-12,1.000000,1242,0.0475,0.0314,0.7746
3,0,59,0,1,7,0,0,49,2010-02-15,0,1,2013-06-12,0.000000,1213,0.0058,0.0486,0.0404
4,1,987,1,1,60,1,0,521,2010-03-15,0,1,2013-06-11,1.000000,1184,0.0507,0.8336,0.4400


### Make "followers_to_friends" ratio
As pointed out in (Z. Chu et al, 2010)  [6], we can often gain more information from engineered features such as a “follower to friends ratio” where humans exhibit a ratio closer to 1, compared to fake followers that tend to 0. This difference could be attributed to bots not posting regularly and not posting organic and engaging content, meaning they garner less followers.

Z. Chu, S. Gianvecchio, H. Wang, and S. Jajodia. Who is tweeting on twitter: Human, bot, or cyborg? In Proceedings of the 26th Annual Computer Security Applications Conference, ACSAC ’10, pages 21–30, New York, NY, USA, 2010. ACM.


In [20]:
df['followers_to_friends'] = df['followers_count']/df['following_count']
# Drop the unwanted columns
df.drop(['followers_count', 'tweet_count', 'following_count', 'created_at', 'updated'], inplace=True, axis=1)
#rearrange the columns so target is in first column
df = df[['target','listed_count', 'location', 'description', 'protected', 'url',
       'verified', 'name_containment', 'account_age_days', 'followers_per_day', 
         'tweets_per_day', 'following_per_day', 'followers_to_friends']]
df.head()

,target,listed_count,location,description,protected,url,verified,name_containment,account_age_days,followers_per_day,tweets_per_day,following_per_day,followers_to_friends
0,1,0,1,1,0,1,0,0.500000,1344,0.0141,0.0216,0.1897,0.074510
1,1,0,0,0,0,0,0,0.333333,1337,0.1556,1.0531,0.6477,0.240185
2,1,0,1,1,0,1,0,1.000000,1242,0.0475,0.0314,0.7746,0.061331
3,1,0,1,0,0,0,0,0.000000,1213,0.0058,0.0486,0.0404,0.142857
4,1,1,1,1,0,1,0,1.000000,1184,0.0507,0.8336,0.4400,0.115163


In [21]:
print("""
    Dimensions
    Rows: {rows}
    Columns: {columns}
    """.format(rows=len(df), columns=len(df.columns)))


    Dimensions
    Rows: 6825
    Columns: 13
    


The final DF contains **7 numerical** features, **5 boolean** features and **1 target**.

In [22]:
# Save the df for the EDA stage
df.to_csv(data_dir+'user_features.csv', index=False)